# Hyperpod Inference with KV cache and Intelligent Routing Admin Notebook

HyperPod's KV Caching and Intelligent Routing features address key challenges in LLM inference, including high latency for long-context prompts, inefficient resource utilization in multi-turn conversations, redundant computation across similar requests, and poor scaling for concurrent users. Our solution combines advanced caching mechanisms with intelligent request routing to deliver an optimized inference experience.

## 1.0 Choose Your Deployment Path

This notebook supports two deployment scenarios. **Choose the path that matches your situation:**

### Path 1: Console Cluster Creation with Inference Operator
**Use this if:** You are creating a new HyperPod cluster
**What it does:** Automatically installs inference operator during cluster creation
**Skip to:** [Section 10.0 - KV Cache & Intelligent Routing Deployment](#10.0)

### Path 2: Install Inference Operator on Existing Cluster
**Use this if:** You have an existing HyperPod cluster without inference operator
**What it does:** Manually installs all required components
**Continue with:** Section 2.0 below

---

## PATH 1: Console Cluster Creation

### 1.1 Create Cluster via AWS Console

This is the simplest path, using the AWS Console with automatic configuration and installation of inference operator and enablement of tiered storage with 20% allocated memory by default.

1. Navigate to the SageMaker AI console
2. Choose HyperPod Cluster from the left navigation pane and select Cluster Management
3. Click Create HyperPod cluster and select Orchestrated by EKS
4. Select Quick Setup
5. In the Storage Configuration section:
   - Enable tiered storage: Check this box
   - Memory allocation percentage: 20% (default, recommended for most workloads)
   - Adjust based on your caching needs (10-40% range)
6. Complete cluster creation
7. Once cluster is ready, **skip to [Section 10.0 - KV Cache & Intelligent Routing Deployment](#endpoint-deployment)**

---

## PATH 2: Install Inference Operator on Existing Cluster

### Prerequisites

This notebook is to be run by Administrator (with Administrator access) plus sagemaker service in the trust policy, and cluster admin access to EKS cluster

For access to EKS cluster:

- Go to EKS console and select the cluster you are using
- Look in the "Access" tab and select "IAM Access Entries"
- If there is not an entry for your execution role:
  - Select "Create Access Entry"
  - Select the desired execution role and correlate the `AmazonEKSClusterAdminPolicy` with the role

**Important:** Ensure the role running this notebook has Admin Access

## 2.0 Set up environment 

### 2.1 Set up environment variables

In [1]:
# Name of your EKS cluster
EKS_CLUSTER_NAME="modelhub16-eks"

# Region
REGION="us-east-1"

# Account Id
ACCOUNT_ID="434444145045"

# Name of Hyperpod cluster
HP_CLUSTER_NAME="modelhub16"

# ARN of Hyperpod cluster
HP_CLUSTER_ARN="arn:aws:sagemaker:us-east-1:434444145045:cluster/kapb14yhzsn7"

# ID of HyperPod cluster (last 12 characters of ARN)
HP_CLUSTER_ID="900cf50a76f6418e83aa1feb1de238b1"

# S3 bucket where tls certificates will be uploaded
TLS_BUCKET_NAME="llm-modelhub-hyperpod-434444145045-us-east-1"

# VPC Id
VPC_ID="llm-modelhub-hyperpod-434444145045-us-east-1"

In [2]:
LB_CONTROLLER_POLICY_NAME = "LBControllerPolicy-" + HP_CLUSTER_ID
S3_MOUNT_ACCESS_POLICY_NAME = "S3MountpointAccessPolicy-" + HP_CLUSTER_ID
S3_CSI_ROLE_NAME = "S3CSIRole-" + HP_CLUSTER_ID
KEDA_OPERATOR_POLICY_NAME = "KedaOperatorPolicy-" + HP_CLUSTER_ID
KEDA_OPERATOR_ROLE_NAME = "KedaOperatorRole-" + HP_CLUSTER_ID
PRESIGNED_URL_ACCESS_POLICY_NAME = "PresignedUrlAccessPolicy" + HP_CLUSTER_ID
HYPERPOD_INFERENCE_ACCESS_POLICY_NAME = "HyperpodInferenceAccessPolicy" + HP_CLUSTER_ID
HYPERPOD_INFERENCE_ROLE_NAME = "HyperpodInferenceRole-" + HP_CLUSTER_ID
HYPERPOD_INFERENCE_SA_NAME="hyperpod-inference-operator-controller"
HYPERPOD_INFERENCE_SA_NAMESPACE="hyperpod-inference-system"
JUMPSTART_GATED_ROLE_NAME = "JumpstartGatedRole-" + HP_CLUSTER_ID
FSX_CSI_ROLE_NAME = "FSxCSIDriverFullAccess-" + HP_CLUSTER_ID

### 2.2 Install dependencies 

Following script downloads and installs Helm, Kubectl and Eksctl if not already installed.

In [ ]:
!git clone https://github.com/aws/sagemaker-hyperpod-cli.git

In [ ]:
!chmod +x sagemaker-hyperpod-cli/helm_chart/install_dependencies.sh
!./sagemaker-hyperpod-cli/helm_chart/install_dependencies.sh

## 3.0 Connect to your EKS Cluster (via kubeconfig)

In [3]:
!aws eks update-kubeconfig --name "$EKS_CLUSTER_NAME" --region "$REGION" --output json

Updated context arn:aws:eks:us-east-1:434444145045:cluster/modelhub16-eks in /home/ubuntu/.kube/config


### 3.1 Confirm Successful Connection to Cluster

The below commands should not show any errors.

In [4]:
!aws sts get-caller-identity

{
    "UserId": "AROAWKJXDSGK5D7ZWT4CB:i-0097bd29342c267b7",
    "Account": "434444145045",
    "Arn": "arn:aws:sts::434444145045:assumed-role/admin_role_for_workshop/i-0097bd29342c267b7"
}


# create-access-entry

For access to EKS cluster with the role above

Go to EKS console and select the cluster you are using.
Note: The correlated EKS cluster name is shown in the output of the above cell.
Look in the "Access" tab and select "IAM Access Entries"
If there is not an entry for your execution role:
Select "Create Access Entry"
Select the desired execution role and correlate the AmazonEKSClusterAdminPolicy with the role

In [5]:
!kubectl config current-context
!kubectl get svc

arn:aws:eks:us-east-1:434444145045:cluster/modelhub16-eks
NAME         TYPE        CLUSTER-IP   EXTERNAL-IP   PORT(S)   AGE
kubernetes   ClusterIP   172.20.0.1   <none>        443/TCP   5d2h


### 3.2 Associate IAM OIDC Provider with EKS Cluster

In [6]:
!eksctl utils associate-iam-oidc-provider --cluster $EKS_CLUSTER_NAME --region $REGION --approve

]11;?\2026-01-12 13:16:32 [ℹ]  IAM Open ID Connect provider is already associated with cluster "modelhub16-eks" in "us-east-1"


## 4.0 Load balancer controller installation

### 4.1 Create IAM Policy for AWSLoadBalancer

In [7]:
!curl -o /tmp/AWSLoadBalancerControllerIAMPolicy.json https://raw.githubusercontent.com/kubernetes-sigs/aws-load-balancer-controller/v2.13.0/docs/install/iam_policy.json
!aws iam create-policy --policy-name $LB_CONTROLLER_POLICY_NAME --policy-document file:///tmp/AWSLoadBalancerControllerIAMPolicy.json

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  8955  100  8955    0     0   118k      0 --:--:-- --:--:-- --:--:--  119k
{
    "Policy": {
        "PolicyName": "LBControllerPolicy-900cf50a76f6418e83aa1feb1de238b1",
        "PolicyId": "ANPAWKJXDSGKS3MCLW74M",
        "Arn": "arn:aws:iam::434444145045:policy/LBControllerPolicy-900cf50a76f6418e83aa1feb1de238b1",
        "Path": "/",
        "DefaultVersionId": "v1",
        "AttachmentCount": 0,
        "PermissionsBoundaryUsageCount": 0,
        "IsAttachable": true,
        "CreateDate": "2026-01-12T13:16:41+00:00",
        "UpdateDate": "2026-01-12T13:16:41+00:00"
    }
}


### 4.2 Create the AWSLoadBalancer Service Account

In [8]:
query = "'Policies[?PolicyName==`" + LB_CONTROLLER_POLICY_NAME + "`].Arn'"
policy_arn_list=!(aws iam list-policies --query $query --output text)

policy_arn=policy_arn_list[0]

!eksctl create iamserviceaccount \
    --approve \
    --override-existing-serviceaccounts \
    --name= "aws-load-balancer-controller" \
    --namespace=kube-system \
    --cluster=$EKS_CLUSTER_NAME \
    --attach-policy-arn=$policy_arn \
    --region $REGION

]11;?\2026-01-12 13:16:54 [ℹ]  1 existing iamserviceaccount(s) (kube-system/aws-load-balancer-controller) will be excluded
2026-01-12 13:16:54 [ℹ]  1 iamserviceaccount (kube-system/aws-load-balancer-controller) was excluded (based on the include/exclude rules)
2026-01-12 13:16:54 [!]  metadata of serviceaccounts that exist in Kubernetes will be updated, as --override-existing-serviceaccounts was set
2026-01-12 13:16:54 [ℹ]  no tasks


### 4.3 Apply Tags to all Subnets in your EKS Cluster (public and private)

In [9]:
filters = "'[{\"Name\":\"vpc-id\", \"Values\": [\"" + VPC_ID + "\"]}, {\"Name\":\"map-public-ip-on-launch\", \"Values\":[\"true\"]}]'"
subnets = !(aws ec2 describe-subnets --filters $filters --output json | jq '.Subnets[]' | jq --raw-output '.SubnetId')
!echo "Applied tags to following subnets"
!echo $subnets
for subnet in subnets:
    !(aws ec2 create-tags --resources $subnet --tags Key=kubernetes.io/role/elb,Value=1)

Applied tags to following subnets
[]


## 5.0 Namespace creation


### 5.1 KEDA Controller Namespace Creation

In [10]:
!kubectl create namespace keda

Error from server (AlreadyExists): namespaces "keda" already exists


### 5.2 Cert Manager Namespace Creation

In [11]:
!kubectl create namespace cert-manager

Error from server (AlreadyExists): namespaces "cert-manager" already exists


### 5.3 Endpoint creation

In [12]:
from IPython.utils.capture import capture_output

with capture_output() as c:
    !(aws ec2 describe-route-tables --region $REGION --filters "Name=vpc-id,Values=$VPC_ID" --query 'RouteTables[].Associations[].RouteTableId' | jq 'unique' | jq -r 'join (" ")')
ROUTE_TABLE_IDS = c.stdout.strip()

SERVICE_NAME="com.amazonaws." + REGION + ".s3"

In [ ]:
# If vpc endpoint is already created via cloudformation, then remove it
import time
import subprocess
from IPython.utils.capture import capture_output

# Check for existing endpoint with corrected query
with capture_output() as c:
    !(aws ec2 describe-vpc-endpoints \
        --region "$REGION" \
        --filters "Name=vpc-id,Values=$VPC_ID" "Name=service-name,Values=$SERVICE_NAME" \
        --query 'VpcEndpoints[?State!=`deleted`].[VpcEndpointId]' \
        --output text)
EXISTING_ENDPOINT = c.stdout.strip()

# If endpoint exists and is not empty/None and only delete S3 VPC endpoint , delete it
if EXISTING_ENDPOINT and EXISTING_ENDPOINT != "None" and "s3" in SERVICE_NAME.lower():
    print("Found existing endpoint, showing details:")
    !(aws ec2 describe-vpc-endpoints --vpc-endpoint-ids "$EXISTING_ENDPOINT" --region "$REGION")

    # Delete the VPC endpoint
    !aws ec2 delete-vpc-endpoints --vpc-endpoint-ids {EXISTING_ENDPOINT} --region {REGION}

    # Wait until the endpoint is fully deleted
    print("Waiting for VPC endpoint to be deleted...")
    SLEEP_INTERVAL = 5
    while True:
        result = subprocess.run(
            ["aws", "ec2", "describe-vpc-endpoints", "--vpc-endpoint-ids", EXISTING_ENDPOINT, "--region", REGION],
            stdout=subprocess.DEVNULL,
            stderr=subprocess.DEVNULL,
        )
        if result.returncode != 0:
            break
        print(".", end="", flush=True)
        time.sleep(SLEEP_INTERVAL)
    print(f"VPC endpoint '{EXISTING_ENDPOINT}' has been successfully deleted.")
else:
    print("No existing endpoint found.")

In [ ]:
!(aws ec2 create-vpc-endpoint --vpc-id $VPC_ID --vpc-endpoint-type "Gateway" --service-name $SERVICE_NAME --route-table-ids $ROUTE_TABLE_IDS --region $REGION)

## 6.0 S3 CSI driver installation

Setup Role for S3 Mountpoint

In [ ]:
%%bash -s "$S3_MOUNT_ACCESS_POLICY_NAME"

cat <<EOF> /tmp/s3-role-policy.json
{
   "Version": "2012-10-17",
   "Statement": [
        {
            "Sid": "MountpointFullBucketAccess",
            "Effect": "Allow",
            "Action": [
                "s3:ListBucket"
            ],
            "Resource": [
                    "arn:aws:s3:::*",
                    "arn:aws:s3:::*/*"
            ]
        },
        {
            "Sid": "MountpointFullObjectAccess",
            "Effect": "Allow",
            "Action": [
                "s3:GetObject",
                "s3:PutObject",
                "s3:AbortMultipartUpload",
                "s3:DeleteObject"
            ],
            "Resource": [
                    "arn:aws:s3:::*",
                    "arn:aws:s3:::*/*"
            ]
        }
   ]
}
EOF

aws iam create-policy --policy-name $1 --policy-document file:///tmp/s3-role-policy.json


In [ ]:
query = "'Policies[?PolicyName==`" + S3_MOUNT_ACCESS_POLICY_NAME + "`].Arn'"
S3_MOUNT_POLICY_ARN_LIST=!(aws iam list-policies --query $query)
S3_MOUNT_POLICY_ARN = S3_MOUNT_POLICY_ARN_LIST[1].strip()

print(S3_MOUNT_POLICY_ARN)

In [ ]:
!eksctl create iamserviceaccount \
    --name s3-csi-driver-sa \
    --override-existing-serviceaccounts \
    --namespace kube-system \
    --cluster $EKS_CLUSTER_NAME \
    --attach-policy-arn $S3_MOUNT_POLICY_ARN \
    --approve \
    --role-name $S3_CSI_ROLE_NAME \
    --region $REGION

In [ ]:
!kubectl label serviceaccount s3-csi-driver-sa app.kubernetes.io/component=csi-driver app.kubernetes.io/instance=aws-mountpoint-s3-csi-driver app.kubernetes.io/managed-by=EKS app.kubernetes.io/name=aws-mountpoint-s3-csi-driver -n kube-system --overwrite

## 6.0.1 Keda operator role creation

In [ ]:
OIDC_ID = !aws eks describe-cluster --name $EKS_CLUSTER_NAME --query "cluster.identity.oidc.issuer" --output text | cut -d '/' -f 5
OIDC_ID = OIDC_ID[0]

In [ ]:
%%bash -s "$REGION" "$OIDC_ID" "$ACCOUNT_ID" "$KEDA_OPERATOR_POLICY_NAME" "$KEDA_OPERATOR_ROLE_NAME"

REGION=$1
OIDC_ID=$2
ACCOUNT_ID=$3
KEDA_OPERATOR_POLICY_NAME=$4
KEDA_OPERATOR_ROLE_NAME=$5

# Create trust policy
cat <<EOF > /tmp/keda-trust-policy.json
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Principal": {
                "Federated": "arn:aws:iam::$ACCOUNT_ID:oidc-provider/oidc.eks.$REGION.amazonaws.com/id/$OIDC_ID"
            },
            "Action": "sts:AssumeRoleWithWebIdentity",
            "Condition": {
                "StringLike": {
                    "oidc.eks.$REGION.amazonaws.com/id/$OIDC_ID:sub": "system:serviceaccount:kube-system:keda-operator",
                    "oidc.eks.$REGION.amazonaws.com/id/$OIDC_ID:aud": "sts.amazonaws.com"
                }
            }
        }
    ]
}
EOF
 
# Create permissions policy
cat <<EOF > /tmp/keda-policy.json
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": [
                "cloudwatch:GetMetricData",
                "cloudwatch:GetMetricStatistics",
                "cloudwatch:ListMetrics"
            ],
            "Resource": "*"
        },
        {
            "Effect": "Allow",
            "Action": [
                "aps:QueryMetrics",
                "aps:GetLabels",
                "aps:GetSeries",
                "aps:GetMetricMetadata"
            ],
            "Resource": "*"
        }
    ]
}
EOF
 
# Create the role
aws iam create-role \
    --role-name $KEDA_OPERATOR_ROLE_NAME \
    --assume-role-policy-document file:///tmp/keda-trust-policy.json
 
# Create the policy
POLICY_ARN=$(aws iam create-policy \
    --policy-name $KEDA_OPERATOR_POLICY_NAME \
    --policy-document file:///tmp/keda-policy.json \
    --query 'Policy.Arn' \
    --output text)
 
# Attach the policy to the role
aws iam attach-role-policy \
    --role-name $KEDA_OPERATOR_ROLE_NAME \
    --policy-arn $POLICY_ARN

In [ ]:
%%bash -s $REGION $PRESIGNED_URL_ACCESS_POLICY_NAME

cat <<EOF> /tmp/presignedurl-policy.json
{
   "Version": "2012-10-17",
   "Statement": [
        {
            "Sid": "CreatePresignedUrlAccess",
            "Effect": "Allow",
            "Action": [
                "sagemaker:CreateHubContentPresignedUrls"
            ],
            "Resource": [
                "arn:aws:sagemaker:$1:aws:hub/SageMakerPublicHub", 
                "arn:aws:sagemaker:$1:aws:hub-content/SageMakerPublicHub/*/*" 
            ]
        }
   ]
}
EOF

aws iam create-policy --policy-name "$2" --policy-document file:///tmp/presignedurl-policy.json

## 7.0 Create execution role

#### 7.1.2 Import the Hyperpod Inference Access Policy to IAM

In [ ]:
!aws iam create-policy --policy-name $HYPERPOD_INFERENCE_ACCESS_POLICY_NAME --policy-document file://inference-resources/hyperpod_inference_operator_policy.json

#### 7.1.3 Create an IAM role with the Hyperpod Inference Access Policy

In [ ]:
query = "'Policies[?PolicyName==`" + HYPERPOD_INFERENCE_ACCESS_POLICY_NAME + "`].Arn'"
policy_arn_list=!(aws iam list-policies --query $query --output text)
policy_arn=policy_arn_list[0]

# Create the IAM role
!eksctl create iamserviceaccount --approve --role-only --name=$HYPERPOD_INFERENCE_SA_NAME --namespace=$HYPERPOD_INFERENCE_SA_NAMESPACE --cluster=$EKS_CLUSTER_NAME --attach-policy-arn=$policy_arn --role-name=$HYPERPOD_INFERENCE_ROLE_NAME --region=$REGION

### 7.2 Presigned URL support

#### 7.2.1 Create role for jumpstart gated model (Optional only for jumpstart gated model)

In [ ]:
JUMPSTART_GATED_ROLE_NAME=f"JumpstartGatedRole-{REGION}-{HP_CLUSTER_NAME}"

In [ ]:
%%bash -s "$REGION" "$OIDC_ID" "$ACCOUNT_ID" "$JUMPSTART_GATED_ROLE_NAME" "$PRESIGNED_URL_ACCESS_POLICY_NAME"

REGION=$1
OIDC_ID=$2
ACCOUNT_ID=$3
JUMPSTART_GATED_ROLE_NAME=$4
PRESIGNED_URL_ACCESS_POLICY_NAME=$5

# Create trust policy
cat <<EOF > /tmp/trust-policy.json
{
	"Version": "2012-10-17",
	"Statement": [
		{
			"Effect": "Allow",
			"Principal": {
				"Federated": "arn:aws:iam::$ACCOUNT_ID:oidc-provider/oidc.eks.$REGION.amazonaws.com/id/$OIDC_ID"
			},
			"Action": "sts:AssumeRoleWithWebIdentity",
			"Condition": {
				"StringLike": {
					"oidc.eks.$REGION.amazonaws.com/id/$OIDC_ID:sub": "system:serviceaccount:*:hyperpod-inference-service-account",
					"oidc.eks.$REGION.amazonaws.com/id/$OIDC_ID:aud": "sts.amazonaws.com"
				}
			}
		}
	]
}
EOF

# Create the role using existing trust policy
aws iam create-role \
    --role-name $JUMPSTART_GATED_ROLE_NAME \
    --assume-role-policy-document file:///tmp/trust-policy.json

# Attach the existing PresignedUrlAccessPolicy to the role
aws iam attach-role-policy \
    --role-name $JUMPSTART_GATED_ROLE_NAME \
    --policy-arn arn:aws:iam::${ACCOUNT_ID}:policy/$PRESIGNED_URL_ACCESS_POLICY_NAME

In [ ]:
JUMPSTART_GATED_ROLE_ARN_LIST= !aws iam get-role --role-name=$JUMPSTART_GATED_ROLE_NAME --query "Role.Arn" --output text
JUMPSTART_GATED_ROLE_ARN = JUMPSTART_GATED_ROLE_ARN_LIST[0]
!echo $JUMPSTART_GATED_ROLE_ARN

#### 7.2.2 Update the Trust Policy

In [ ]:
%%bash -s "$HYPERPOD_INFERENCE_ROLE_NAME" "$HYPERPOD_INFERENCE_SA_NAMESPACE" "$HYPERPOD_INFERENCE_SA_NAME"

cat <<EOF> hyperpod_inference_additional_trust_statement.txt
        {
            "Effect": "Allow",
            "Principal": {
                "Service": "sagemaker.amazonaws.com"
            },
            "Action": "sts:AssumeRole"
        },
EOF

aws iam get-role --role-name=$1 --query "Role"."AssumeRolePolicyDocument" > hyperpod_inference_trust_policy.json
if [[ $(diff hyperpod_inference_additional_trust_statement.txt hyperpod_inference_trust_policy.json | grep '^<' -) ]]; then
    echo Trust policy missing some permissions. Updating with the permissions above.
    { head -n3 hyperpod_inference_trust_policy.json && cat hyperpod_inference_additional_trust_statement.txt && tail -n+4 hyperpod_inference_trust_policy.json; } > /tmp/updated_hyperpod_inference_trust_policy.json
    # Also allow any service account to AssumeRoleWithWebIdentity
    sed -i '0,/"StringEquals": {/{s//"StringLike": {/}' /tmp/updated_hyperpod_inference_trust_policy.json
    sed -i "0,/:sub\": \"system:serviceaccount:${2}:${3}\"/{s//:sub\": \"system:serviceaccount:*:*\"/}" /tmp/updated_hyperpod_inference_trust_policy.json
    aws iam update-assume-role-policy --role-name=$1 --policy-document=file:///tmp/updated_hyperpod_inference_trust_policy.json
else
    echo Trust policy contains all necessary permissions.
fi

### 7.4 Get role arns to be used during installation of operator

In [ ]:
HYPERPOD_INFERENCE_ROLE_ARN_list= !(aws iam get-role --role-name=$HYPERPOD_INFERENCE_ROLE_NAME --query "Role.Arn" --output text)
HYPERPOD_INFERENCE_ROLE_ARN = HYPERPOD_INFERENCE_ROLE_ARN_list[0]
!echo $HYPERPOD_INFERENCE_ROLE_ARN

In [ ]:
S3_CSI_ROLE_ARN_LIST= !aws iam get-role --role-name=$S3_CSI_ROLE_NAME --query "Role.Arn" --output text
S3_CSI_ROLE_ARN = S3_CSI_ROLE_ARN_LIST[0]
!echo $S3_CSI_ROLE_ARN

## 8.0 Install operator

In [ ]:
HELM_CMD="helm install hyperpod-inference-operator ./sagemaker-hyperpod-cli/helm_chart/HyperPodHelmChart/charts/inference-operator \
     -n kube-system \
     --set region=" + REGION + " \
     --set eksClusterName=" + EKS_CLUSTER_NAME + " \
     --set hyperpodClusterArn=" + HP_CLUSTER_ARN + " \
     --set executionRoleArn=" + HYPERPOD_INFERENCE_ROLE_ARN + " \
     --set s3.serviceAccountRoleArn=" + S3_CSI_ROLE_ARN + " \
     --set s3.node.serviceAccount.create=false \
     --set keda.podIdentity.aws.irsa.roleArn=\"arn:aws:iam::" + ACCOUNT_ID + ":role/keda-operator-role\" \
     --set tlsCertificateS3Bucket=" + TLS_BUCKET_NAME + " \
     --set alb.region=" + REGION + " \
     --set alb.clusterName=" + EKS_CLUSTER_NAME + " \
     --set alb.vpcId=" + VPC_ID + " \
     --set jumpstartGatedModelDownloadRoleArn=" + JUMPSTART_GATED_ROLE_ARN


In [ ]:
!echo $HELM_CMD

In [ ]:
!helm dependencies update sagemaker-hyperpod-cli/helm_chart/HyperPodHelmChart/charts/inference-operator

In [ ]:
!eval $HELM_CMD

## 9.0 FSx CSI service account update

In [ ]:
 !eksctl create iamserviceaccount \
       --name fsx-csi-controller-sa \
       --override-existing-serviceaccounts \
       --namespace kube-system \
       --cluster $EKS_CLUSTER_NAME \
       --attach-policy-arn arn:aws:iam::aws:policy/AmazonFSxFullAccess \
       --role-name $FSX_CSI_ROLE_NAME \
       --approve \
       --region $REGION 

## 10.0 KV Cache and Intelligent Routing Deployment

This section provides a complete example of deploying an inference endpoint with KV caching and intelligent routing capabilities.

### 10.1 Prerequisites for Endpoint Deployment

Before proceeding with the deployment, ensure your environment meets the following requirements:

- **AWS CLI**: Configured with appropriate IAM permissions for EKS, S3, and SageMaker operations
- **kubectl**: Properly configured to communicate with your target EKS cluster
- **Helm**: Helm installed for managing Kubernetes applications
- **Repository Access**: Clone the SageMaker HyperPod CLI repository from GitHub
- **Model Access**: Valid Hugging Face token for accessing gated models 

Verify your cluster has sufficient instances.

a) Begin by cloning the repository and navigating to the inference operator directory. 

In [ ]:
!git clone https://github.com/aws/sagemaker-hyperpod-cli.git

b) Determine the namespace of the 'hyperpod-inference-operator' Helm deployment. More often than not this should be 'kube-system'.

In [13]:
!helm list -A | grep hyperpod-inference-operator

hyperpod-inference-operator	kube-system	1       	2026-01-07 11:03:30.304922715 +0000 UTC	deployed	hyperpod-inference-operator-1.1.0	2.1        


c) Pipe the values used for setup for an existing cluster to a file

In [ ]:
!helm get values -n $DEPLOYMENT_NAMESPACE hyperpod-inference-operator -o yaml > $DESIRED_FILE_PATH

d) Run 'helm upgrade' using the existing values and the force flag.

In [ ]:
!helm dependency update sagemaker-hyperpod-cli/helm_chart/HyperPodHelmChart/charts/inference-operator/

!helm upgrade -n $DEPLOYMENT_NAMESPACE hyperpod-inference-operator sagemaker-hyperpod-cli/helm_chart/HyperPodHelmChart/charts/inference-operator/ -f $DESIRED_FILE_PATH --force

### 10.2 S3 Model Upload

We are using Llama-3.1-8B-Instruct. Obtain the model from Hugging Face (authentication may be required for gated models)
and ensure the model directory structure includes all necessary files.
After downloading, upload the complete model directory to your S3 bucket.

In [ ]:
# Upload the entire model directory to S3
!aws s3 cp <your-model-file> <s3://bucket-name/path/>

# Verify upload completed
!aws s3 ls s3://your-bucket-name/models/Llama-3.1-8B-Instruct/ --recursive


After the model upload is complete and you've verified all files are accessible in S3, proceed with endpoint creation.

### 10.3 Create an Endpoint
Specify the name and namespace of the endpoint in the metadata section.
Specify your S3 bucketname, region and S3 Uri based on where the model location.

In [ ]:
%% endpoint-config.yaml
apiVersion: inference.sagemaker.aws.amazon.com/v1
kind: InferenceEndpointConfig
metadata:
  name: demo
  namespace: default
spec:
  endpointName: demo-l2-cache   #include endpoint name
  modelName: Qwen3-4B-Instruct   #include model name 
  instanceType: ml.g5.4xlarge          #include desired instance type
  invocationEndpoint: v1/chat/completions
  replicas: 1
  modelSourceConfig:
    modelSourceType: s3
    s3Storage:
      bucketName: sagemaker-us-east-1-434444145045  #include your bucket name
      region: us-east-1                  #include your region
    modelLocation: s3://sagemaker-us-east-1-434444145045/Qwen3-4B-Instruct-2507/0d0f748bad7a4aa7a3cae5ec42bb4553/finetuned_model_merged/   #include model location 
    prefetchEnabled: false
  kvCacheSpec:
    enableL1Cache: true
    enableL2Cache: true
    l2CacheSpec:
     l2CacheBackend: tieredstorage
  intelligentRoutingSpec:
    enabled: true
    routingStrategy: prefixaware
  tlsConfig:
    tlsCertificateOutputS3Uri: s3://llm-modelhub-hyperpod-434444145045-us-east-1/cert   #include your S3URI
  metrics:
    enabled: true
    modelMetrics:
      port: 8000
  loadBalancer:
    healthCheckPath: /health
  worker:
    resources:
      limits:
        nvidia.com/gpu: "1"
      requests:
        cpu: "6"
        memory: 30Gi
        nvidia.com/gpu: "1"
    image: public.ecr.aws/deep-learning-containers/vllm:0.13.0-gpu-py312-cu129-ubuntu22.04-ec2-v1.0
    args:
      - "/opt/ml/model"
      - "--max-model-len"
      - "20000"
      - "--tensor-parallel-size"
      - "1"
    modelInvocationPort:
      containerPort: 8000
      name: http
    modelVolumeMount:
      name: model-weights
      mountPath: /opt/ml/model
    environmentVariables:
      - name: PYTHONHASHSEED
        value: "123"
      - name: OPTION_ROLLING_BATCH
        value: "vllm"
      - name: SAGEMAKER_SUBMIT_DIRECTORY
        value: "/opt/ml/model/code"
      - name: MODEL_CACHE_ROOT
        value: "/opt/ml/model"
      - name: SAGEMAKER_MODEL_SERVER_WORKERS
        value: "1"
      - name: SAGEMAKER_MODEL_SERVER_TIMEOUT
        value: "3600"

UsageError: Cell magic `%%` not found.


#### 10.3.1 Enable Intelligent Routing
If you want to enable intelligent routing, you can do that by making the intelligentRoutingSpec as true and you can disable it by marking it false

In [ ]:
intelligentRoutingSpec:
    enabled: true  #CHANGE HERE
    routingStrategy: prefixaware

##### 10.3.1.1 Change the Routing Strategy

The intelligent routing system supports four different strategies that can be configured based on your requirements: "prefixaware", "kvaware", "session" , and "roundrobin".

* **Prefix-aware routing (default)**: Maintains a tree structure to track which prefixes are cached on which endpoints, making it effective for workloads with common prompt prefixes. While it provides good general-purpose performance, it cannot detect when cache entries are evicted from workers, potentially leading to suboptimal routing decisions.
* **Round-robin routing**: It is the most straightforward approach, which brute-force distributes requests evenly across all available workers. While simple to implement, it doesn't optimize for cache reuse and is best suited for scenarios where requests are independent and cache sharing isn't critical.
* **Session-based routing**: Ensures requests from the same session are consistently routed to the same worker, making it ideal for maintaining conversation context in chatbot applications. However, it limits cache sharing opportunities across different sessions, potentially leading to redundant cache entries across workers.
* **KV-aware routing**: Offers the most sophisticated cache management by using a centralized controller to track cache locations and handle cache eviction events. However, it requires tokenizing prompts in the router and currently only supports Hugging Face models. This strategy is best when precise cache control is needed and these limitations are acceptable.

This can be done using the "routingStrategy" parameter.

In [ ]:
intelligentRoutingSpec:
    enabled: true  
    routingStrategy: prefixaware  #CHANGE HERE- "prefixaware", "kvaware", "session" , and "roundrobin"

For "session" and "kvaware" routing strategies, you can customize the session identifier by adding a "SESSION_KEY" environment variable to your endpoint configuration.


In [ ]:
environmentVariables:
  - name: SESSION_KEY           
    value: "user_id"

#### 10.3.2 Enable L1 Cache

**L1 Cache**: Local CPU memory cache on each node for fastest access to recently computed key-value pairs.

If you want to enable L1 cache, you can do that by making the kvCacheSpec true and you can disable it by marking it false

In [ ]:
kvCacheSpec:
    enableL1Cache: true  #CHANGE HERE

#### 10.3.3 Enable L2 Cache

**L2Cache**: Remote storage like Redis that enables offloading intermediate states across multiple nodes for shared access to key-value pairs.

To enable L2 Cache functionality, ensure the Redis service is running and accessible on the configured port.
 
**Important:** The Redis cluster must be deployed within the same VPC as your HyperPod cluster to ensure network connectivity and maintain data encryption in transit. Verify the Redis pod is running before creating the endpoint.

##### 10.3.3.1 redis.yaml

In [ ]:
apiVersion: v1
kind: ConfigMap
metadata:
  name: redis-config
data:
  redis.conf: |
    # --- Memory ---
    maxmemory 75gb
    maxmemory-policy allkeys-lfu
    # Pure cache mode
    appendonly no
    save ""

    # --- Concurrency / networking ---
    io-threads 8
    io-threads-do-reads yes
    tcp-backlog 65535
    tcp-keepalive 300
    timeout 0

    # Protect server at high fan-in (slow consumers, giant replies)
    client-output-buffer-limit normal 128mb 64mb 60
    client-output-buffer-limit replica 256mb 64mb 60
    client-output-buffer-limit pubsub 32mb 8mb 60
    client-query-buffer-limit 256mb
    proto-max-bulk-len 256mb

    # Allow many clients (match to file-descriptor ulimit below)
    maxclients 200000

    # Misc latency hygiene
    dynamic-hz yes
    hz 100


---
# Or keep a standard ClusterIP if you prefer (comment one of the Services out)
apiVersion: v1
kind: Service
metadata:
  name: redis
spec:
  selector:
    app: redis
  ports:
  - name: redis
    port: 6379
    targetPort: 6379
  type: ClusterIP

---
apiVersion: apps/v1
kind: StatefulSet
metadata:
  name: redis
spec:
  serviceName: redis-headless
  replicas: 1
  selector:
    matchLabels: { app: redis }
  template:
    metadata:
      labels: { app: redis }
      annotations:
        # Skip service meshes/proxies if present
        sidecar.istio.io/inject: "false"
    spec:
      # Fastest path (optional). Remove these two lines if you don't want host networking.
      hostNetwork: true
      dnsPolicy: ClusterFirstWithHostNet

      # Keep Redis on beefy, quiet nodes
      nodeSelector:
        node.kubernetes.io/instance-type: ml.m5.24xlarge

      # Prefer co-location policy (tweak to your topology/keyspace)
      topologySpreadConstraints:
      - maxSkew: 1
        topologyKey: topology.kubernetes.io/zone
        whenUnsatisfiable: ScheduleAnyway
        labelSelector:
          matchLabels:
            app: redis

      containers:
      - name: redis
        image: redis:7.2-alpine
        imagePullPolicy: IfNotPresent
        args: ["redis-server", "/usr/local/etc/redis/redis.conf"]
        env:
          - name: PYTHONHASHSEED
            value: "123"
        ports:
        - containerPort: 6379
          name: redis
        resources:
          # Reserve real cores; **omit CPU limits** to prevent throttling
          requests:
            cpu: "32"
            memory: "90Gi"
          # Remove limits entirely for best latency, or keep a high memory limit if you must
          # limits:
          #   memory: "100Gi"
        readinessProbe:
          tcpSocket: { port: 6379 }
          initialDelaySeconds: 3
          periodSeconds: 3
        livenessProbe:
          tcpSocket: { port: 6379 }
          initialDelaySeconds: 10
          periodSeconds: 10
        volumeMounts:
        - name: cfg
          mountPath: /usr/local/etc/redis
      volumes:
      - name: cfg
        configMap:
          name: redis-config

Apply the above file using kubectl apply, after the pod is running, we can start with enabling L2.

##### 10.3.3.2 Enable L2 Cache Field 

In [ ]:
enableL2Cache: true     #Change here
l2CacheSpec:
    l2CacheBackend: redis
    l2CacheLocalUrl: redis://redis.default.svc.cluster.local:6379

The l2CacheLocalUrl mentioned above should follow this format

In [ ]:
<redis-service-name>.<namespace>.svc.cluster.local:6379

### 10.4 Apply the File
To apply the above file, use the following command

In [17]:
!kubectl apply -f endpoint-config.yaml

inferenceendpointconfig.inference.sagemaker.aws.amazon.com/demo created


### 10.5 View the Pods

In [23]:
!kubectl get inferenceendpointconfig -n default  # List all inference endpoints
!kubectl describe inferenceendpointconfig demo -n default  # Show detailed deployment info

NAME                                      AGE
demo                                      7m24s
qwen3-4b-instruct-2507-2026-01-13-06-42   6m34s
Name:         demo
Namespace:    default
Labels:       <none>
Annotations:  <none>
API Version:  inference.sagemaker.aws.amazon.com/v1
Kind:         InferenceEndpointConfig
Metadata:
  Creation Timestamp:  2026-01-13T06:41:38Z
  Finalizers:
    inference.sagemaker.aws.InferenceEndpointConfigFinalizer
    inference.sagemaker.aws.amazon.com/metrics-configmap
  Generation:        2
  Resource Version:  2552047
  UID:               166060b7-96cc-49c7-8813-1da6158a2d0a
Spec:
  Auto Scaling Spec:
    Cloud Watch Trigger:
      Activation Target Value:       0
      Metric Collection Period:      300
      Metric Collection Start Time:  300
      Metric Stat:                   Average
      Metric Type:                   Average
      Min Value:                     0
      Use Cached Metrics:            true
    Cooldown Period:                 300
    

When the pod status shows "Running", the deployment is complete and ready to serve inference requests.

### 10.6 Invoke the Endpoint

In [32]:
!pip install boto3 --break-system-packages

Defaulting to user installation because normal site-packages is not writeable


  Using cached jmespath-1.0.1-py3-none-any.whl.metadata (7.6 kB)
  Using cached s3transfer-0.16.0-py3-none-any.whl.metadata (1.7 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 126.7 MB/s eta 0:00:0000:0100:01
Using cached jmespath-1.0.1-py3-none-any.whl (20 kB)
Using cached s3transfer-0.16.0-py3-none-any.whl (86 kB)


In [ ]:
import boto3
import json

endpoint_name = "demo-l2-cache"
# endpoint_name = "qwen3-4b-instruct-2507-2026-01-13-06-05"
runtime = boto3.client("sagemaker-runtime", region_name="us-east-1")

payload = {
         "model": "/opt/ml/model",
            "messages": [
                {
                    "role": "user",
                    "content": "What is machine learning?"
                }
            ],
            "max_tokens": 50,
            "temperature": 0,
            "user_id": "session_123"  # Add this field for "session" and "kvaware" routing
        }
response = runtime.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType="application/json",
    Body=json.dumps(payload)
)

print(response["Body"].read().decode())

ModelError: An error occurred (ModelError) when calling the InvokeEndpoint operation: Received client error (400) from primary with message "{"error":"Model /opt/ml/model not found or vLLM engine is sleeping."}". See https://us-east-1.console.aws.amazon.com/cloudwatch/home?region=us-east-1#logEventViewer:group=/aws/sagemaker/Endpoints/demo-l2-cache in account 434444145045 for more information.

## 11 Uninstallation

In [ ]:
!helm uninstall hyperpod-inference-operator -n kube-system

## 12 Conclusion

KV Cache and Intelligent Routing in SageMaker HyperPod Model Deployment help you optimize LLM inference performance and costs through efficient memory management and smart request routing. You can get started today by adding these configurations to your HyperPod model deployments in all AWS Regions where SageMaker HyperPod is available.